In [1]:
import numpy as np
import pandas as pd
import unicodedata
import re
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


In [2]:
data = pd.read_csv('output.csv')


In [3]:
# 1. Loại bỏ khoảng trắng và điền NaN
data['attributes'] = data['attributes'].str.strip()
data['attributes'] = data['attributes'].fillna('Không rõ')  # Loại bỏ inplace=True

# 2. Mã hóa các thuộc tính
attributes_dummies = data['attributes'].str.get_dummies(sep=', ')

# 3. Ghép các cột đã mã hóa vào DataFrame gốc
data = pd.concat([data, attributes_dummies], axis=1)

# 4. Kiểm tra kết quả
print(data.head())



   province  district                   street  price_unit  price_value  \
0         1         5  Mặt phố C1 TT Nghĩa Tân           1         5.50   
1         1         5                  Yên Hoà           1         8.58   
2         1         5                 Cầu Giấy           1         8.65   
3         1         5              Hồ Tùng Mậu           1        28.50   
4         1         5                 Cầu Giấy           1         8.90   

   price_absolute   area  type  front  room  toilet  floor  \
0          5500.0   70.0     1    7.0     2       2      1   
1          8580.0   45.0     1    5.0     5       4      4   
2          8650.0   43.0     1    3.6     6       1      5   
3         28500.0  100.0     1    6.9    29       0      8   
4          8900.0   35.0     1    3.4     6       2      5   

                    attributes  Chung cư  Dòng tiền  Không rõ  Kinh doanh  \
0  Ô tô, Kinh doanh, Dòng tiền         0          1         0           1   
1                     

In [4]:
 # Hàm chuẩn hóa tên đường
def normalize_street_name(street_name):
    # Loại bỏ dấu tiếng Việt
    street_name = unicodedata.normalize('NFKD', street_name).encode('ascii', 'ignore').decode('utf-8')
    # Chuyển thành chữ thường
    street_name = street_name.lower()
    # Loại bỏ các ký tự đặc biệt và khoảng trắng thừa
    street_name = re.sub(r'[^a-z0-9\s]', '', street_name)
    street_name = re.sub(r'\s+', ' ', street_name).strip()
    return street_name

# Áp dụng hàm chuẩn hóa lên cột 'street'
data['street_normalized'] = data['street'].apply(normalize_street_name)

# Thực hiện one-hot encoding sau khi chuẩn hóa
street_encoded = pd.get_dummies(data['street_normalized'], prefix='street')

# Nối cột mới với DataFrame gốc
data = pd.concat([data, street_encoded], axis=1)

# Xóa cột 'street_normalized' nếu không cần thiết nữa
data.drop(columns=['street_normalized'], inplace=True)

# Kiểm tra lại dữ liệu sau khi xử lý
print(data.head())

   province  district                   street  price_unit  price_value  \
0         1         5  Mặt phố C1 TT Nghĩa Tân           1         5.50   
1         1         5                  Yên Hoà           1         8.58   
2         1         5                 Cầu Giấy           1         8.65   
3         1         5              Hồ Tùng Mậu           1        28.50   
4         1         5                 Cầu Giấy           1         8.90   

   price_absolute   area  type  front  room  ...  street_truong cong giai  \
0          5500.0   70.0     1    7.0     2  ...                    False   
1          8580.0   45.0     1    5.0     5  ...                    False   
2          8650.0   43.0     1    3.6     6  ...                    False   
3         28500.0  100.0     1    6.9    29  ...                    False   
4          8900.0   35.0     1    3.4     6  ...                    False   

   street_tt van cong mai dich street_tt van hoa nghe thuat mai dich  \
0             

In [5]:
data = data.drop(columns=['street'])

data = data.drop(columns=['attributes'])

data = data.drop(columns=['price_value'])

X = data.drop('price_absolute', axis=1)
y = data['price_absolute']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [8]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import pandas as pd

In [42]:
# Tăng cường grid các tham số
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.2],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.05, 0.1, 0.2, 0.3, 0.4],
    'min_child_weight': [1, 2, 3, 4, 5]
}


In [34]:
# Khởi tạo mô hình XGBRegressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror'
, random_state=101)

In [35]:
# Tăng số lần thử nghiệm và số lượng folds
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=100,  # Tăng số lượng lần thử nghiệm
    scoring='neg_mean_squared_error',
    cv=8,  # Số lượng cross-validation folds tăng lên 7
    verbose=1,
    n_jobs=-1,
    random_state=101
)

In [36]:

# Huấn luyện với RandomizedSearchCV
random_search.fit(X_train, y_train)


Fitting 8 folds for each of 100 candidates, totalling 800 fits


RandomizedSearchCV(cv=8,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                   n_iter=100, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.5, 0.6, 0.7, 0.8,
                                                             0.9, 1.0],
                                        'gamma': [0, 0.05, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.005, 0.01, 0.05,
                                                          0.1, 0.2],
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                                        'min_child_weight': [1, 2, 3, 4, 5],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9,
                                                      1.0]},
                   random_state=101, scoring='neg_mean_squared_error',
                   verbose=1)

In [37]:
# Lấy ra các tham số tốt nhất
best_params = random_search.best_params_
print(f"Best Parameters: {best_params}")

Best Parameters: {'subsample': 0.5, 'n_estimators': 300, 'min_child_weight': 4, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.5}


In [38]:
# Huấn luyện lại mô hình với các tham số tốt nhất
best_xgb_model = random_search.best_estimator_


In [39]:
# Dự đoán trên tập kiểm tra
y_test_pred = best_xgb_model.predict(X_test)

In [40]:
# Tính toán các metric
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
rmse = root_mean_squared_error(y_test, y_test_pred)  # Sử dụng hàm mới cho RMSE
r2 = r2_score(y_test, y_test_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

MSE: 13564343140.107769
MAE: 22683.77199016378
RMSE: 116466.06003513542
R2 Score: 0.349974714326897


In [43]:
# Tạo DataFrame so sánh
df_comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
df_comparison['Absolute Error'] = abs(df_comparison['Actual'] - df_comparison['Predicted'])

# Tính toán MAPE cho từng mẫu
df_comparison['Percentage Error'] = abs((df_comparison['Actual'] - df_comparison['Predicted']) / df_comparison['Actual']) * 100

# Tính toán MAPE tổng quát
mape = df_comparison['Percentage Error'].mean()

# Hiển thị 10 kết quả so sánh đầu tiên
print("10 kết quả so sánh đầu tiên:")
print(df_comparison.head(10))

# Hiển thị MAPE
print(f"\nMean Absolute Percentage Error (MAPE): {mape:.2f}%")

10 kết quả so sánh đầu tiên:
      Actual     Predicted  Absolute Error  Percentage Error
220  17500.0  16925.128906      574.871094          3.284978
18   16500.0  17246.605469      746.605469          4.524882
391   7550.0   9051.669922     1501.669922         19.889668
141  44000.0  27444.175781    16555.824219         37.626873
354  14500.0  24226.892578     9726.892578         67.082018
116  26500.0  22340.023438     4159.976562         15.698025
313  13500.0  14871.138672     1371.138672         10.156583
138  22500.0  15435.713867     7064.286133         31.396827
515  25500.0  18525.253906     6974.746094         27.351945
360   8600.0  11189.297852     2589.297852         30.108115

Mean Absolute Percentage Error (MAPE): inf%
